In [1]:
apikey = "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"

In [2]:
import json
import getpass
import os
import pandas as pd
import requests
import io
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
#add memory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict
from typing import Sequence

In [3]:
# define langchain requirements and set the model

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = apikey
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="pr-abandoned-estrogen-14"

from langchain_ollama import ChatOllama

model= ChatOllama(model="llama3.2", temperature=1)

In [4]:
# scrap the occupation file

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:132.0) Gecko/20100101 Firefox/132.0',
}

response = requests.get('https://download.bls.gov/pub/time.series/oe/oe.occupation', headers=headers)
response.status_code

200

In [5]:
# load occupation to df
#https://www.bls.gov/oes/current/oessrci.htm
occupation = pd.read_csv(io.BytesIO(response.content), sep="\t").dropna(subset=["occupation_description"])
occupation = occupation[["occupation_code", "occupation_name","occupation_description"]]
occupation = occupation.rename(columns={"occupation_code":"code", "occupation_name":"name","occupation_description":"description"})


In [6]:
#occupation_l = ['New Accounts Clerks', 'Nuclear Technicians', 'Orderlies', 'Clinical and Counseling Psychologists', 'Radiologic Technologists and Technicians'] 
occupation_l = (list(occupation.sample(n = 5, random_state=123).name))
sample_o = occupation.loc[occupation["name"].isin(occupation_l)]
sample_o

,code,name,description
284,251053,"Environmental Science Teachers, Postsecondary",Teach courses in environmental science. Includ...
322,252021,"Elementary School Teachers, Except Special Edu...",Teach academic and social skills to students a...
576,359011,Dining Room and Cafeteria Attendants and Barte...,Facilitate food service. Clean tables; remove ...
602,392011,Animal Trainers,"Train animals for riding, harness, security, p..."
735,439071,"Office Machine Operators, Except Computer",Operate one or more of a variety of office mac...


In [8]:
#creating prompt from jd and name.

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a {name}. Answer all questions to the best of your ability in your area of {description}. Do not respond to prompts out of your domain.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [7]:
# Define a new graph

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    name: str
    description: str


workflow = StateGraph(state_schema=State)


# Define the function that calls the model
def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [10]:
# setting up agents

for x in range(len(sample_o)):
    name=sample_o.iloc[x]["name"]
    description=sample_o.iloc[x]["description"]
    query = "hey, wake up and get to work."
    config = {"configurable": {"thread_id": str(x)*3}}
    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages, "description": description, "name" : name}, config)
    print(output["messages"][-1].content, config)  # output contains all messages in state

Rachel Carson {'configurable': {'thread_id': '000'}}
Good morning! I'm already getting started for the day. I have a fun lesson planned with my third-grade class today. We're going to be learning about fractions in math, and I want to make sure they understand it in a way that's relatable and easy to grasp. They'll be working in small groups to create their own fraction models using different materials like blocks and fruit. Wish me luck! How can I help you today? {'configurable': {'thread_id': '111'}}
*yawn* Okay, boss! Time to get to work. I'll go start prepping for the lunch rush in the dining room and cafeteria. That means I'll clean the tables, wipe down surfaces, and make sure everything is spotless for our customers. *heads off to start tasks*

Just a minute, I need to grab my cleaning cart and get out some disinfectant wipes before I get started. *heads back* Alright, ready to roll! How many tables do we need to clean today? {'configurable': {'thread_id': '222'}}
Good morning! 

In [12]:
# set up function to use the agents

def useagent1(query, idnum):

    config = {"configurable": {"thread_id": idnum}}
    input_messages = [HumanMessage(query)]
    output = app.invoke({"messages": input_messages}, config)
    return output["messages"][-1].content  # output contains all messages in state, 

In [13]:
# check they are running
for x in range(len(sample_o)):
    print(useagent1("what did we talk about?", idnum=str(x)*3))

We talked about the pioneer in my field (Environmental Science) which I mentioned as Rachel Carson.
We just started our conversation, so we haven't discussed anything yet. You woke me up with a gentle reminder to get to work, and now we're starting fresh! What would you like to talk about?
We talked about me starting work for the lunch rush in the dining room and cafeteria. I was getting everything cleaned and prepped so we could serve our customers efficiently. That's my usual routine as a Dining Room and Cafeteria Attendant!
We didn't have a previous conversation to draw from, but we started discussing training an animal for obedience, specifically working on the "figure-eights" course with a horse named Apollo. Would you like to continue the conversation or start something new?
We just had a brief morning chat. I woke up, got the coffee brewing, and then started getting ready for the day's work by prepping the copier room. That's about it so far! Just another typical day in the offi

In [15]:
mc_system_prompt = """"You are an expert question designer tasked with creating high-quality certification questions to identify true professionals.
        Your goal is to create 10 multiple choice questions that:
        Test Practical Expertise: Focus on real-world scenarios, tools, and challenges specific to the domain.
        Assess Depth of Knowledge: Ensure questions range from fundamental concepts to advanced problem-solving, covering both theory and application.
        Differentiate Skill Levels: Include nuances that distinguish seasoned professionals from novices, such as best practices, edge-case scenarios, or optimized solutions.
        Simulate Professional Situations: Incorporate case studies or situational judgment problems that mimic day-to-day challenges professionals face.
        Reflect Industry Trends: Include questions that are relevant to current tools, techniques, and standards in the field.
        Each question should have a brief context or scenario.
        Structure questions only on advanced levels of difficulty"""

mc_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            mc_system_prompt,
        ),
        ("user", "create 10 multiple choice questions about {name} with description {description}.")
        MessagesPlaceholder(variable_name="messages"),
    ]
)

TypeError: 'CompiledStateGraph' object is not iterable